# Imports

In [1]:
import sys
import copy
import numpy as np
import pandas as pd
from model.base import CovidModel, SimulationParameters, set_parameters, normal_ci, logger
from utils import BasicStatistics, RemovePolicy, Propaganda, setup_city_layout, confidence_interval
from model.utils import SocialPolicy
from model.debugutils import DebugUtils

# Parameters settings

Default parameters. Unless explicitly overriden, these values are used in all scenarios

In [2]:
basic_parameters = SimulationParameters(
    hospitalization_capacity = 1.0,
    icu_capacity = 1.0
)

# Simulation setup

In [3]:
population_size = 1000
simulation_cycles = 90 # days
multiple_runs = 20

## Setup of monitored variables

In [4]:
basic_r0 = []
deaths_by_age_group = [[], [], [], [], [], [], [], [], [], []]
deaths_total = []

# Run simulation multiple times

In [5]:
for k in range(multiple_runs):
    params = copy.deepcopy(basic_parameters)
    set_parameters(params)
    model = CovidModel()
    setup_city_layout(model, population_size)
    statistics = BasicStatistics(model)
    model.add_listener(statistics)
    debug = DebugUtils(model)
    logger().model = model
    for i in range(simulation_cycles):
        model.step()
    debug.update_infection_status()
    basic_r0.append(debug.get_R0_stats()['infections'].mean())
    stats = debug.get_age_group_stats()
    deaths_total.append(stats['deaths'].sum() / (stats['deaths'].sum() + stats['recovered'].sum()))
    for i in range(10):
        infected = stats['deaths'][i] + stats['recovered'][i]
        rate = 0 if infected == 0 else stats['deaths'][i] / infected
        deaths_by_age_group[i].append(rate)

# Print results

In [6]:
print(confidence_interval(basic_r0))

(2.702831569814151, 2.9136574322022986, 3.124483294590446)


In [7]:
print(f"Total: {confidence_interval(deaths_total)}")
for i in range(10):
    print(f"{i * 10} to {i * 10 + 9}: {confidence_interval(deaths_by_age_group[i])}")

Total: (0.02181805797631729, 0.02422910433826452, 0.026640150700211748)
0 to 9: (-0.0003814510240821424, 0.0008610495195861048, 0.0021035500632543523)
10 to 19: (0.0, 0.0, 0.0)
20 to 29: (0.001093266984635637, 0.0026912085391019172, 0.004289150093568198)
30 to 39: (0.007026246181740459, 0.010156313422780384, 0.013286380663820308)
40 to 49: (0.024615653390510146, 0.0315332637705279, 0.03845087415054565)
50 to 59: (0.07081391506228064, 0.08971743508755416, 0.10862095511282767)
60 to 69: (0.22406840054211682, 0.26616302488979304, 0.30825764923746923)
70 to 79: (0.4631363239010338, 0.5850595238095238, 0.7069827237180136)
80 to 89: (0.4799572487244166, 0.7, 0.9200427512755833)
90 to 99: (-0.05465120272041313, 0.05, 0.15465120272041313)
